## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Изучение общей информации о базе данных.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/datasets/data.csv')

In [3]:
# изучим общую информацию:
#df.head()
df.info()
#df.duplicated().sum()
#df['children'].value_counts()
#df[df['days_employed']<0]['days_employed'].count()
#df[-df['days_employed']>(df['dob_years']-14)*365].count()
#df['dob_years'].value_counts()
#df[df['dob_years'] < 14]['dob_years'].count()
#df[df['dob_years'] > 120]['dob_years'].count()
#df['education'].value_counts()
#df['education_id'].value_counts()
#df['family_status'].value_counts()
#df['family_status_id'].value_counts()
#df['gender'].value_counts()
#df['income_type'].value_counts()
#df[df['total_income']<0]['total_income'].count()
#df['debt'].value_counts()
#df['purpose'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

При изучении таблицы df было обнаружено, что:
- Имеются явные пропуски значений 'days_employed' и 'total_income' для ряда заемщиков.
- Таблица содержит дупликаты строк.
- Имееются негативные (-1) и явно ошибочные (20) значения 'children'.
- Имeются негативные и явно ошибочные (стаж больше возраста за вычетом 14 лет) значения 'days_employed'.
- 'days_employed' и 'total_income' представлены с излишним количеством знаков после запятой.
- Имеются явно ошибочные (<14) значения 'dob_years'.
- Одинаковые значения 'education' записаны в разных регистрах.
- При записи значений 'family_status' использованы различные регистры.
- Столбец 'gender' в одной из строк содержит явно ошибочное значение 'XNA'.
- 'purpose' содержит смысловые дупликаты.

### Шаг 1а. Обработка дупликатов строк

Т.к. в дальнейшем предполагается обработка пропусков с использованием медианных значений, то необходимо предварительно обработать дупликаты строк.

In [4]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

### Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
# сначала обработаем пропуски в 'dob_years'. хотя явных пропусков в этом столбце обнаружено не было, но имеются значения
# менее 14 лет.заменим эти значения на средний возраст заемщиков.

# средний возраст заемщиков за исключением пенсионеров:
mean_age = df[(df['dob_years']>=14)&(df['income_type']!='пенсионер')]['dob_years'].mean()
# средний возраст пенсионеров:
mean_age_rtd = df[(df['dob_years']>=14)&(df['income_type']=='пенсионер')]['dob_years'].mean()
# заменим 'dob_years'<14 непенсионеров на средний возраст:
df.loc[(df['dob_years']<14)&(df['income_type']!='пенсионер'),'dob_years'] = df[df['dob_years']<14]['dob_years'].replace(df['dob_years'],mean_age)
# заменим 'dob_years'<14 пенсионеров на средний возраст:
df.loc[(df['dob_years']<14)&(df['income_type']=='пенсионер'),'dob_years'] = df[df['dob_years']<14]['dob_years'].replace(df['dob_years'],mean_age_rtd)
# перейдем к целочисленным значениям 'dob_years':
df['dob_years'] = df['dob_years'].astype('int64')

# обработаем пропуски в 'days_employed':

# перейдем к абсолютным (положительным значениям) 'days_employed':
df['days_employed']=pd.to_numeric(df['days_employed'],errors='coerce')
df['days_employed']=df['days_employed'].abs()
# заменим пропущенные и явно ошибочные (превышающие возраст за вычетом 14 лет) значения стажа на 0:
df.loc[(df['days_employed']>((df['dob_years']-14)*365)),'days_employed'] = df[df['days_employed']>(df['dob_years']-14)*365]['days_employed'].replace(df['days_employed'],0)
df['days_employed']=df['days_employed'].fillna(0)

# заменим значения стажа =0 на среднее значение для соответствующего возраста:
for row in range(len(df)):
    age = df.loc[row,'dob_years']
    de = df.loc[row,'days_employed']
    max_de = (age-14)*365
    if de == 0:
        mean_de = df[(df['dob_years']==age)&(df['days_employed']<=max_de)]['days_employed'].mean()
        df.at[row,'days_employed'] = mean_de        

# обработаем пропуски в 'total_income'
        
# медианный доход заемщиков за исключением пенсионеров:
median_income = df[df['income_type']!='пенсионер']['total_income'].median()
# медианный доход пенсионеров:
median_income_rtd = df[df['income_type']=='пенсионер']['total_income'].median()
# заменим пропуски 'total_income' непенсионеров на медианный доход:
df.loc[df['income_type']!='пенсионер','total_income'] = df[df['income_type']!='пенсионер']['total_income'].fillna(median_income)
# заменим пропуски 'total_income' пенсионеров на медианный доход:
df.loc[df['income_type']=='пенсионер','total_income'] = df[df['income_type']=='пенсионер']['total_income'].fillna(median_income_rtd)

# обработаем пропуски в 'children'. хотя явных пропусков в этом столбце обнаружено не было, но имеется 47 отрицательных (-1)
# и 76 явно ошибочных (20) значений.

# перейдем к абсолютым (положительным значениям) 'children':
df['children'] = df['children'].abs()
# удалим строки таблицы с явно ошибочными (20) значениями 'children':
df = df[df['children'] != 20]

# обработаем пропуски в 'gender' - удалим строку со значением 'XNA' (такая строка одна):
df = df[df['gender'] != 'XNA']

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21394 entries, 0 to 21470
Data columns (total 12 columns):
children            21394 non-null int64
days_employed       21394 non-null float64
dob_years           21394 non-null int64
education           21394 non-null object
education_id        21394 non-null int64
family_status       21394 non-null object
family_status_id    21394 non-null int64
gender              21394 non-null object
income_type         21394 non-null object
debt                21394 non-null int64
total_income        21394 non-null float64
purpose             21394 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Вывод

В этом разделе мы обработали пропуски и явно ошибочные значения данных. При обработке использовали следующие допущения:
- Минимальный возраст заключения трудового и кредитного договора не может быть менее 14 лет.
- Количество заемщиков типов занятости "безработный", "предприниматель", "студент" и "в декрете" пренебрежимо мало (2, 2, 1 и 1 соответсвенно) в сравнении с общим количеством наблюдений и может быть рассмотрено в одной группе с другими типами занятости за исключением пенсионеров.
- Средний возраст и медианный доход пенсионеров существенно отличается от заемщиков других типов занятости.
- Явно ошибочные значения возраста были заменены на средние значения (непенсионеры и пенсионеры отдельно).
- Явно ошибочные и пропущенные значения стажа были заменены на средние значения для соответсвующего возраста.
- Пропущенные значения дохода были заменены на медианные значения (непенсионеры и пенсионеры отдельно).
- Отрицательные значения количества детей являются ошибочной записью соответствующей положительной величины.
- Количество детей =20 у 76 заемщиков явно не соответсвует статистическим данным и, вероятно, является опечаткой клавиш 2 и 0. Т.к. факт наличия детей является существенной информацией для решения данной задачи, то было решено удалить строки с данной неопределенностью. Количество удаленных строк составляет несущественную долю от общего числа наблюдений.
- Значение пола 'XNA' было решено считать пропуском, строка (она одна) была удалена.

### Замена типа данных

In [6]:
# заменим вещественный тип данных на целочисленный:
df['days_employed'] = df['days_employed'].astype('int64')
df['total_income'] = df['total_income'].astype('int64')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,2184,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Перевод значений в целочисленный тип выполнили методом astype(), т.к. начальный тип значений был float24. Если бы имелись строковые значения, то применили бы метод pd.to_numeric(). Значения 'dob_years' перевели в целочисленный вид выше, в разделе "Обработка пропусков".

### Обработка (смысловых) дубликатов

In [7]:
# переведем значениия 'education' в нижний регистр:
df['education'] = df['education'].str.lower()
# проверим 'education' на наличие смысловых дупликатов:
#df['education'].value_counts()
# смысловых дупликатов в 'education' не обнаружено.

# переведем значениия 'family_status' в нижний регистр:
df['family_status'] = df['family_status'].str.lower()
# проверим 'family_status' на наличие смысловых дупликатов:
#df['family_status'].value_counts()
# смысловых дупликатов в 'family_status' не обнаружено.

# переведем значениия 'gender' в нижний регистр:
df['gender'] = df['gender'].str.lower()
# проверим 'gender' на наличие смысловых дупликатов:
df['gender'].value_counts()
# смысловых дупликатов в 'gender' не обнаружено.

# переведем значениия 'income_type' в нижний регистр:
df['income_type'] = df['income_type'].str.lower()
# проверим 'income_type' на наличие смысловых дупликатов:
#df['income_type'].value_counts()
# смысловых дупликатов в 'income_type' не обнаружено.

# переведем значения 'purpose' в нижний регистр:
df['purpose'] = df['purpose'].str.lower()
# проверим 'purpose' на наличие смысловых дупликатов:
df['purpose'].value_counts()
# в 'purpose' обнаружено множество смысловых дупликатов.

свадьба                                   792
на проведение свадьбы                     769
сыграть свадьбу                           765
операции с недвижимостью                  674
покупка коммерческой недвижимости         659
покупка жилья для сдачи                   651
операции с жильем                         648
операции с коммерческой недвижимостью     646
покупка жилья                             643
жилье                                     642
покупка жилья для семьи                   637
недвижимость                              632
строительство собственной недвижимости    629
операции со своей недвижимостью           626
строительство жилой недвижимости          624
покупка своего жилья                      620
строительство недвижимости                619
покупка недвижимости                      618
ремонт жилью                              605
покупка жилой недвижимости                604
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

От дупликатов строк избавились выше, в разделе 1а. Смысловые дупликаты обработаем методом лемматизации в следующем разделе.

### Лемматизация

In [8]:
from pymystem3 import Mystem
m = Mystem()

def fl(text):
    lemma = m.lemmatize(text)
    if 'свадьба' in lemma:
        return 'свадьба'
    elif 'жилье' in lemma:
        return 'недвижимость'
    elif 'недвижимость' in lemma:
        return 'недвижимость'
    elif 'автомобиль' in lemma:
        return 'автомобиль'
    elif 'образование' in lemma:
        return 'образование'
    else:
        return 'другое'
df['purpose_category']=df['purpose'].apply(fl)
df['purpose_category'].value_counts()
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,образование
4,0,2184,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,свадьба


### Вывод

Для удаления смысловых дупликатов, обнаруженных ранее в 'purpose', мы воспользовались методом лемматизации. Для этого создали функцию fl с аргументом text и применили её методом apply() к столбцу 'purpose'.
Примечание: В 1-й версии работы леммитазацию сделал через циклы. Тоже работало. Но так лучше.

### Категоризация данных

In [9]:
# введем классификацию по количеству детей в семье:
def children_status (children):
    if children == 0:
        return 'нет детей'
    if children >= 3:
        return '3 или более'
    return '1-2 ребенка'

# введем классификацию по уровню дохода - 8 медианных группы:
max = df['total_income'].max()
min = df['total_income'].min()
med = df['total_income'].median().astype('int64')
med1 = df[df['total_income']<=med]['total_income'].median().astype('int64')
med1a = df[df['total_income']<=med1]['total_income'].median().astype('int64')
med1b = df[(df['total_income']>med1)&(df['total_income']<=med)]['total_income'].median().astype('int64')
med2 = df[df['total_income']>med]['total_income'].median().astype('int64')
med2a = df[(df['total_income']>med)&(df['total_income']<=med2)]['total_income'].median().astype('int64')
med2b = df[df['total_income']>med2]['total_income'].median().astype('int64')
def income_range (income):
    if income < med1a:
        return (min,med1a)
    elif income < med1:
        return (med1a,med1)
    elif income < med1b:
        return (med1,med1b)
    elif income < med:
        return (med1b,med)
    elif income < med2a:
        return (med,med2a)
    elif income < med2:
        return (med2a,med2)
    elif income < med2b:
        return (med2,med2b)
    return (med2b,max)

# добавим столбцы в таблицу df:
df['children_status'] = df['children'].apply(children_status)
df['income_range'] = df['total_income'].apply(income_range)

# поменяем порядок столбцов в таблице:
titles = ['children_status','family_status','income_range','purpose_category','debt']
df1 = df.reindex(columns=titles)
df1.head(10)

,children_status,family_status,income_range,purpose_category,debt
0,1-2 ребенка,женат / замужем,"(209291, 265758)",недвижимость,0
1,1-2 ребенка,женат / замужем,"(87536, 114055)",автомобиль,0
2,нет детей,женат / замужем,"(138137, 151134)",недвижимость,0
3,3 или более,женат / замужем,"(265758, 2265604)",образование,0
4,нет детей,гражданский брак,"(151134, 174631)",свадьба,0
5,нет детей,гражданский брак,"(209291, 265758)",недвижимость,0
6,нет детей,женат / замужем,"(209291, 265758)",недвижимость,0
7,нет детей,женат / замужем,"(114055, 138137)",образование,0
8,1-2 ребенка,гражданский брак,"(87536, 114055)",свадьба,0
9,нет детей,женат / замужем,"(138137, 151134)",недвижимость,0


### Вывод

- Провели классификацию по количеству детей в семье - "нет детей", "1-2 ребенка, "3 или более".
- По признаку семейного положения категории представлены вполне логично, поэтому классификацию проводить не стали.
- Уровень дохода классифицировали по 8-и медианным категориям.
- Категории цели кредита были введены на этапе лемматизации.
- Для удобства дальнейшей работы убрали ненужные более столбцы и поменяли оставшиеся столбцы местами. Получили таблицу df1. С ней и будем работать далее.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [10]:
# здесь и далее критерием будем считать разность долей от общего числа клиентов и от общего числа должников.
# чем меньше критерий, тем меньше благонадежность.

# общее количество должников:
debt_ttl = df1[df1['debt'] == 1]['debt'].count()

# зависимость между наличием детей и невозвратом кредита в срок: 

columns1 = ['children_status','debtors_%','clients_%', 'criteria_%']
data1 = []
cs_list = df1['children_status'].unique() #список категорий возраста детей
for i in cs_list:
    cs = df1[df1['children_status'] == i]['children_status'].count()/len(df1)*100
    cs_d = df1[(df1['children_status'] == i) & (df1['debt'] == 1)]['debt'].count()/debt_ttl*100
    cs_x = cs-cs_d
    line1 = [i,cs_d,cs, cs_x]
    data1.append(line1)
table1 = pd.DataFrame(data = data1, columns=columns1)
table1=table1.round(0)
table1.sort_values('criteria_%')


,children_status,debtors_%,clients_%,criteria_%
0,1-2 ребенка,37.0,32.0,-5.0
2,3 или более,2.0,2.0,-0.0
1,нет детей,61.0,66.0,5.0


### Вывод

Наименее благонадежны заемщики, имеющие 1-2 ребенка.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [11]:
columns2 = ['family_status','debtors_%','clients_%', 'criteria_%']
data2 = []
fs_list = df1['family_status'].unique() #список категорий семейного положения
for i in fs_list:
    fs = df1[df1['family_status'] == i]['family_status'].count()/len(df1)*100
    fs_d = df1[(df1['family_status'] == i) & (df1['debt'] == 1)]['debt'].count()/debt_ttl*100
    fs_x = fs-fs_d
    line2 = [i,fs_d,fs,fs_x]
    data2.append(line2)
table2 = pd.DataFrame(data = data2, columns=columns2)
table2 = table2.round(0)
table2.sort_values('criteria_%')

,family_status,debtors_%,clients_%,criteria_%
1,гражданский брак,22.0,19.0,-3.0
4,не женат / не замужем,16.0,13.0,-3.0
2,вдовец / вдова,4.0,4.0,1.0
3,в разводе,5.0,6.0,1.0
0,женат / замужем,54.0,57.0,4.0


### Вывод

Наименее благонадежны неженатые/незамужние заемщики и состоящие в гражданском браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [12]:
columns3 = ['income_range','debtors_%','clients_%', 'criteria_%']
data3 = []
ir_list = df1['income_range'].unique() #список категорий уровня дохода
for i in ir_list:
    ir = df1[df1['income_range'] == i]['income_range'].count()/len(df1)*100
    ir_d = df1[(df1['income_range'] == i) & (df1['debt'] == 1)]['income_range'].count()/debt_ttl*100
    ir_x = ir - ir_d
    line3 = [i,ir_d,ir,ir_x]
    data3.append(line3)
table3 = pd.DataFrame(data = data3, columns=columns3)
table3 = table3.round(0)
table3.sort_values('criteria_%')

,income_range,debtors_%,clients_%,criteria_%
1,"(87536, 114055)",15.0,14.0,-1.0
2,"(138137, 151134)",7.0,6.0,-1.0
4,"(151134, 174631)",19.0,19.0,-1.0
5,"(114055, 138137)",15.0,14.0,-1.0
7,"(174631, 209291)",11.0,11.0,-0.0
0,"(209291, 265758)",9.0,11.0,1.0
6,"(20667, 87536)",14.0,14.0,1.0
3,"(265758, 2265604)",9.0,11.0,2.0


### Вывод

Наименее благонадежны заемщики с уровнем дохода 88-175 т.р., но значение критерия незначительно.

- Как разные цели кредита влияют на его возврат в срок?

In [13]:
columns4 = ['purpose_category','debtors_%','clients_%','criteria_%']
data4 = []
pc_list = df1['purpose_category'].unique() #список категорий цели кредита
for i in pc_list:
    pc = df1[df1['purpose_category'] == i]['purpose_category'].count()/len(df1)*100
    pc_d = df1[(df1['purpose_category'] == i) & (df1['debt'] == 1)]['purpose_category'].count()/debt_ttl*100
    pc_x = pc-pc_d
    line4 = [i,pc_d,pc,pc_x]
    data4.append(line4)
table4 = pd.DataFrame(data = data4, columns=columns4)
table4=table4.round(0)
table4.sort_values('criteria_%')

,purpose_category,debtors_%,clients_%,criteria_%
1,автомобиль,23.0,20.0,-3.0
2,образование,21.0,19.0,-3.0
3,свадьба,11.0,11.0,0.0
0,недвижимость,45.0,50.0,5.0


### Вывод

Наименее благонадежны заемщики, берущие кредит на покупку автомобиля, далее по степени риска идут лица, берущие кредит на образование.

### Шаг 4. Общий вывод

Вывод о благонадежности делался путем сравнения доли определенной категории заемщиков в общем числе должников с долей этой категории от общего числа заемщиков.
Сравнительно менее благонадежны следующие категории заемщиков (в скобках указана разность долей):
- имеющие 1-2 ребенка (5%),
- неженатые/незамужние заемщики, за исключением разведенных и вдовцов (3%), 
- состоящие в гражданском браке (3%),
- с уровнем дохода 88-175 т.р. (1%),
- берущие кредит на приобретение автомобиля (3%),
- берущие кредит на образование (3%).

Бросается в глаза незначительная зависимость вероятности невыплаты кредита в срок от уровня дохода заемщика. Это может быть связано с суммой кредита, предоставленного банком.
